# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
vaca_data = pd.read_csv("../output_data/cities.csv", dtype = "object", encoding="utf-8")

vaca_data.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.5,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.7,78.8,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.2,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.6,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:

# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [5]:
locations = vaca_data[["Lat", "Lng"]].astype(float)

humidty = vaca_data["Humidity"].astype(float)

vaca_data.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.5,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.7,78.8,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.2,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.6,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [6]:
fig = gmaps.figure( zoom_level = 2, center = (-33.93,18.42))

heat_layer = gmaps.heatmap_layer(locations, weights=humidty * 50, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
vaca_data.dtypes

vaca_max_temp = vaca_data.loc[(vaca_data['Max Temp'].astype(float) <= 80) & (vaca_data['Max Temp'].astype(float) > 70)]
vaca_wind = vaca_max_temp.loc[(vaca_max_temp['Wind Speed'].astype(float) <= 10)]
vaca_cloudiness = vaca_wind.loc[(vaca_wind['Cloudiness'].astype(float) <= 4)]

vaca_ideal_df = vaca_cloudiness.dropna()
vaca_ideal_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.0,73.68,5.32
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.5
138,138,morondava,0,MG,1558378780,69,-20.3,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.9,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.2,6.93
268,268,dingle,4,PH,1558378836,80,11.0,122.67,79.08,2.73
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.6,9.17
494,494,baylo,1,PH,1558378889,69,7.85,126.06,77.0,1.07


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
hotel_df = vaca_ideal_df
hotel_df["Hotel Name"] = ""
hotel_df


    

/Users/davidruppel/opt/anaconda3/envs/bcs/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.0,73.68,5.32,
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52,
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.5,
138,138,morondava,0,MG,1558378780,69,-20.3,44.28,79.08,9.06,
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.9,1.59,
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.2,6.93,
268,268,dingle,4,PH,1558378836,80,11.0,122.67,79.08,2.73,
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.6,9.17,
494,494,baylo,1,PH,1558378889,69,7.85,126.06,77.0,1.07,


In [30]:
target_search = "Hotel"
target_radius = 5000
target_type = "hotel"


# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for i,r in hotel_df.iterrows():
    target_lat = r['Lat']
    target_lon = r['Lng']
    target_coordinates = f'{target_lat}, {target_lon}'
    
    # set up a parameters dictionary
    params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    # convert response to json
    places_data = response.json()

    # Print the json (pretty printed)
    try:
        print(places_data['results'][0]['name'])
        hotel_df.loc[i,'Hotel Name'] = places_data['results'][0]['name']
    except IndexError:
        print("Hotel not found")
        


Escarpas Resort


/Users/davidruppel/opt/anaconda3/envs/bcs/lib/python3.6/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/davidruppel/opt/anaconda3/envs/bcs/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Hotel National
Pousada Vila Di Italy
Palissandre Cote Ouest resort & SPA
Dandeli Lake County
Hotel Marchica Lagoon Resort, Nador Morocco
Hotel not found
Mount Mogok Hotel
Ghaem Hotel
Hotel not found


In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]].astype(float)

In [37]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=5,
    info_box_content=hotel_info
)

# Display figure
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [34]:
hotel_df.dtypes

City_ID       object
City          object
Cloudiness    object
Country       object
Date          object
Humidity      object
Lat           object
Lng           object
Max Temp      object
Wind Speed    object
Hotel Name    object
dtype: object